# Week 2: Identify Nearest Health Facilities

<span style="color:red">
**UPDATE**

Thank you for your analysis. Despite our warning efforts so far, the virus continues to spread rapidly. We want to get infected individuals treatment as quickly as possible, so we need your help to calculate which hospital or clinic is closest to each known infected individual in the population.
</span>

Your goal for this notebook will be to identify the nearest hospital or clinic for each infected person.

## Imports

In [1]:
import cudf
import cuml
import cupy as cp

## Load Population Data

Begin by loading the `lat`, `long` and `infected` columns from `'./data/week2.csv'` into a cuDF data frame called `gdf`.

In [2]:
gdf = cudf.read_csv('./data/week2.csv', usecols = ['lat','long', 'infected'])
gdf

,lat,long,infected
0,54.522510,-1.571896,0.0
1,54.554030,-1.524968,0.0
2,54.552486,-1.435203,0.0
3,54.537189,-1.566215,0.0
4,54.528212,-1.588462,0.0
...,...,...,...
58479889,51.634416,-2.925863,0.0
58479890,51.556972,-3.036290,0.0
58479891,51.588992,-2.921915,0.0
58479892,51.590974,-2.954539,0.0


## Load Hospital and Clinics Data

For this step, your goal is to create an `all_med` cuDF data frame that contains the latitudes and longitudes of all the hospitals (data found at `'./data/hospitals.csv'`) and clinics (data found at `'./data/clinics.csv'`).

In [3]:
all_med = cudf.read_csv('./data/hospitals.csv')
all_med.append(cudf.read_csv('./data/clinics.csv'))

,﻿OrganisationID,OrganisationCode,OrganisationType,SubType,Sector,OrganisationStatus,IsPimsManaged,OrganisationName,Address1,Address2,...,County,Postcode,Latitude,Longitude,ParentODSCode,ParentName,Phone,Email,Website,Fax
0,17970,NDA07,Hospital,Hospital,Independent Sector,Visible,True,Walton Community Hospital - Virgin Care Servic...,<NA>,Rodney Road,...,Surrey,KT12 3LD,51.379997,-0.406042,NDA,Virgin Care Services Ltd,01932 414205,<NA>,<NA>,01932 253674
1,17981,NDA18,Hospital,Hospital,Independent Sector,Visible,True,Woking Community Hospital (Virgin Care),<NA>,Heathside Road,...,Surrey,GU22 7HS,51.315132,-0.556289,NDA,Virgin Care Services Ltd,01483 715911,<NA>,<NA>,<NA>
2,18102,NLT02,Hospital,Hospital,NHS Sector,Visible,True,North Somerset Community Hospital,North Somerset Community Hospital,Old Street,...,Avon,BS21 6BS,51.437195,-2.847193,NLT,North Somerset Community Partnership Community...,01275 872212,<NA>,http://www.nscphealth.co.uk,<NA>
3,18138,NMP01,Hospital,Hospital,Independent Sector,Visible,False,Bridgewater Hospital,120 Princess Road,<NA>,...,Greater Manchester,M15 5AT,53.459743,-2.245469,NMP,Bridgewater Hospital (Manchester) Ltd,0161 2270000,<NA>,www.bridgewaterhospital.com,<NA>
4,18142,NMV01,Hospital,Hospital,Independent Sector,Visible,True,Kneesworth House,Old North Road,Bassingbourn,...,<NA>,SG8 5JP,52.078121,-0.030604,NMV,Partnerships In Care Ltd,01763 255 700,reception_kneesworthhouse@partnershipsincare.c...,www.partnershipsincare.co.uk,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19077,10617742,NBG7L,Clinic,UNKNOWN,<NA>,Visible,False,Sunnybank Medical Centre,Town Gate,Wyke,...,<NA>,BD12 9NG,53.737923,-1.767982,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
19078,10617743,NFH02,Clinic,UNKNOWN,<NA>,Visible,False,Somerset Surgical Services At Mendip Vale Medi...,Pudding Pie Lane,Langford,...,<NA>,BS40 5EL,51.341579,-2.784808,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
19079,10617744,NFH03,Clinic,UNKNOWN,<NA>,Visible,False,Somerset Surgical Services At Axbridge Medical...,Houlgate Way,<NA>,...,<NA>,BS26 2BJ,51.286285,-2.820388,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
19080,10617746,NEMAM,Clinic,UNKNOWN,<NA>,Visible,False,Lawton House Surgery,Bromley Road,<NA>,...,<NA>,CW12 1QG,53.163048,-2.205781,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


Since we will be using the coordinates of those facilities, keep only those rows that are non-null in both  `Latitude` and `Longitude`.

In [4]:
all_med.dropna(subset=['Latitude','Longitude'], inplace= True)

## Make Grid Coordinates for Medical Facilities

Provided for you in the next cell (which you can expand by clicking on the "...", and contract again after executing by clicking on the blue left border of the cell) is the lat/long to grid coordinates converter you have used earlier in the workshop. Use this converter to create grid coordinate values stored in `northing` and `easting` columns of the `all_med` data frame you created in the last step.

In [5]:
# https://www.ordnancesurvey.co.uk/docs/support/guide-coordinate-systems-great-britain.pdf

def latlong2osgbgrid_cupy(lat, long, input_degrees=True):
    '''
    Converts latitude and longitude (ellipsoidal) coordinates into northing and easting (grid) coordinates, using a Transverse Mercator projection.
    
    Inputs:
    lat: latitude coordinate (N)
    long: longitude coordinate (E)
    input_degrees: if True (default), interprets the coordinates as degrees; otherwise, interprets coordinates as radians
    
    Output:
    (northing, easting)
    '''
    
    if input_degrees:
        lat = lat * cp.pi/180
        long = long * cp.pi/180

    a = 6377563.396
    b = 6356256.909
    e2 = (a**2 - b**2) / a**2

    N0 = -100000 # northing of true origin
    E0 = 400000 # easting of true origin
    F0 = .9996012717 # scale factor on central meridian
    phi0 = 49 * cp.pi / 180 # latitude of true origin
    lambda0 = -2 * cp.pi / 180 # longitude of true origin and central meridian
    
    sinlat = cp.sin(lat)
    coslat = cp.cos(lat)
    tanlat = cp.tan(lat)
    
    latdiff = lat-phi0
    longdiff = long-lambda0

    n = (a-b) / (a+b)
    nu = a * F0 * (1 - e2 * sinlat ** 2) ** -.5
    rho = a * F0 * (1 - e2) * (1 - e2 * sinlat ** 2) ** -1.5
    eta2 = nu / rho - 1
    M = b * F0 * ((1 + n + 5/4 * (n**2 + n**3)) * latdiff - 
                  (3*(n+n**2) + 21/8 * n**3) * cp.sin(latdiff) * cp.cos(lat+phi0) +
                  15/8 * (n**2 + n**3) * cp.sin(2*(latdiff)) * cp.cos(2*(lat+phi0)) - 
                  35/24 * n**3 * cp.sin(3*(latdiff)) * cp.cos(3*(lat+phi0)))
    I = M + N0
    II = nu/2 * sinlat * coslat
    III = nu/24 * sinlat * coslat ** 3 * (5 - tanlat ** 2 + 9 * eta2)
    IIIA = nu/720 * sinlat * coslat ** 5 * (61-58 * tanlat**2 + tanlat**4)
    IV = nu * coslat
    V = nu / 6 * coslat**3 * (nu/rho - cp.tan(lat)**2)
    VI = nu / 120 * coslat ** 5 * (5 - 18 * tanlat**2 + tanlat**4 + 14 * eta2 - 58 * tanlat**2 * eta2)

    northing = I + II * longdiff**2 + III * longdiff**4 + IIIA * longdiff**6
    easting = E0 + IV * longdiff + V * longdiff**3 + VI * longdiff**5

    return(northing, easting)

In [6]:
med_lat = cp.asarray(all_med['Latitude'])
med_long = cp.asarray(all_med['Longitude'])
northing, easting = latlong2osgbgrid_cupy(med_lat, med_long)
all_med['northing'], all_med['easting'] = northing, easting

## Find Closest Hospital or Clinic for Infected

Fit `cuml.NearestNeighbors` with `all_med`'s `northing` and `easting` values, using the named argument `n_neighbors` set to `1`, and save the model as `knn`.

In [7]:
knn = cuml.NearestNeighbors(n_neighbors = 1)
knn.fit(all_med[['northing','easting']])

NearestNeighbors()

Save every infected member in `gdf` into a new dataframe called `infected_gdf`.

In [8]:
infected_gdf = gdf.loc[gdf['infected'] == 1]

Create `northing` and `easting` values for `infected_gdf`.

In [9]:
infected_gdf_lat = infected_gdf['lat']
infected_gdf_long = infected_gdf['long']
northing, easting= latlong2osgbgrid_cupy(infected_gdf_lat,infected_gdf_long)
infected_gdf['northing'], infected_gdf['easting'] = northing, easting
infected_gdf

,lat,long,infected,northing,easting
1346586,53.715826,-2.430079,1.0,424489.783814,371619.678741
1350932,53.664881,-2.425673,1.0,418820.687944,371876.492369
1352085,53.696765,-2.488940,1.0,422394.398940,367721.000265
1352799,53.696966,-2.488897,1.0,422416.821887,367723.973098
1357529,53.727804,-2.392959,1.0,425808.109929,374076.557677
...,...,...,...,...,...
58459189,51.662717,-2.926850,1.0,196451.131712,335900.971171
58459551,51.599350,-2.959175,1.0,189433.449408,333573.002864
58461935,51.543825,-2.822984,1.0,183143.984735,342935.130365
58464082,51.562536,-2.879492,1.0,185270.296666,339042.062616


Use `knn.kneighbors` with `n_neighbors=1` on `infected_gdf`'s `northing` and `easting` values. Save the return values in `distances` and `indices`.

In [10]:
distances, indices = knn.kneighbors(infected_gdf[['northing','easting']], 1)

RuntimeError: CUDA error encountered at: file=_deps/raft-src/cpp/include/raft/spatial/knn/detail/knn_brute_force_faiss.cuh line=355: call='cudaPeekAtLastError()', Reason=cudaErrorIllegalAddress:an illegal memory access was encountered
Obtained 64 stack frames
#0 in /opt/conda/envs/rapids/lib/python3.8/site-packages/cuml/common/../../../../libcuml++.so(_ZN4raft9exception18collect_call_stackEv+0x3b) [0x7ff1e3c2f54b]
#1 in /opt/conda/envs/rapids/lib/python3.8/site-packages/cuml/common/../../../../libcuml++.so(_ZN4raft10cuda_errorC1ERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE+0xbd) [0x7ff1e3c30a8d]
#2 in /opt/conda/envs/rapids/lib/python3.8/site-packages/cuml/common/../../../../libraft_nn.so(_ZN4raft7spatial3knn6detail20brute_force_knn_implIjlfEEvRKNS_8handle_tERSt6vectorIPT1_SaIS9_EERS7_IT_SaISD_EESD_S9_SD_PT0_S9_SD_bbPS7_ISH_SaISH_EENS_8distance12DistanceTypeEf+0x1ffd) [0x7ff1bfd1208d]
#3 in /opt/conda/envs/rapids/lib/python3.8/site-packages/cuml/common/../../../../libraft_nn.

### Check Your Solution

`indices`, returned from your use of `knn.kneighbors` immediately above, should map person indices to their closest clinic/hospital indices:

In [11]:
indices.head()

NameError: name 'indices' is not defined

Here you can print an infected individual's coordinates from `infected_gdf`:

In [ ]:
infected_gdf.iloc[0] # get the coords of an infected individual (in this case, individual 0)

You should be able to used the mapped index for the nearest facility to see that indeed the nearest facility is at a nearby coordinate:

In [ ]:
all_med.iloc[1234] # printing the entry for facility 1234 (replace with the index identified as closest to the individual)

<div align="center"><h2>Please Restart the Kernel</h2></div>

...before moving to the next notebook.

In [12]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

Exception ignored in: 'cupy.cuda.function.Module.__dealloc__'
Traceback (most recent call last):
  File "cupy_backends/cuda/api/driver.pyx", line 260, in cupy_backends.cuda.api.driver.moduleUnload
  File "cupy_backends/cuda/api/driver.pyx", line 125, in cupy_backends.cuda.api.driver.check_status
cupy_backends.cuda.api.driver.CUDADriverError: CUDA_ERROR_ILLEGAL_ADDRESS: an illegal memory access was encountered
